In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

DEFAULT_SEED = 42

from gaussian import MultivariateNormal, VarianceExploding, VariancePreserving, SubVariancePreserving
from numerical import EulerSolver, BroydenSolver
from utils import cube_vertices, plot_simulation, solve_numerical_scheme, solve_flow

In [ ]:
norms = cube_vertices(dim=2)

### Explicit Euler plots

In [ ]:
mix = VarianceExploding(norms)

for linear_ts in [False, True]:
    t, x, _ = solve_numerical_scheme(solver=EulerSolver, mix=mix, n_samples=5000, t_min=1e-6, tf=2.0, n_timesteps=500, linear_ts=linear_ts)
    plot_simulation(mix, t, x, show_every=120)

In [ ]:
mix = VariancePreserving(norms)

for linear_ts in [False, True]:
    t, x, _ = solve_numerical_scheme(solver=EulerSolver, mix=mix, n_samples=5000, t_min=1e-6, tf=1.0, n_timesteps=500, linear_ts=linear_ts)
    plot_simulation(mix, t, x, show_every=120)

In [ ]:
mix = SubVariancePreserving(norms)

for linear_ts in [False, True]:
    t, x, _ = solve_numerical_scheme(solver=EulerSolver, mix=mix, n_samples=5000, t_min=1e-6, tf=1.0, n_timesteps=500, linear_ts=linear_ts)
    plot_simulation(mix, t, x, show_every=120)

### Implicit Euler (using Broyden method) plots

In [ ]:
mix = VarianceExploding(norms)

for linear_ts in [False, True]:
    t, x, _ = solve_numerical_scheme(solver=BroydenSolver, mix=mix, n_samples=5000, t_min=1e-6, tf=2.0, n_timesteps=500, linear_ts=linear_ts)
    plot_simulation(mix, t, x, show_every=120)

In [ ]:
mix = VariancePreserving(norms)

for linear_ts in [False, True]:
    t, x, _ = solve_numerical_scheme(solver=BroydenSolver, mix=mix, n_samples=5000, t_min=1e-6, tf=1.0, n_timesteps=500, linear_ts=linear_ts)
    plot_simulation(mix, t, x, show_every=120)

In [ ]:
mix = SubVariancePreserving(norms)

for linear_ts in [False, True]:
    t, x, _ = solve_numerical_scheme(solver=BroydenSolver, mix=mix, n_samples=5000, t_min=1e-6, tf=1.0, n_timesteps=500, linear_ts=linear_ts)
    plot_simulation(mix, t, x, show_every=120)

### Negative log-likelihood per formulation (VE, VP, sub-VP)

In [ ]:
tf = 2.0
mix = VarianceExploding(norms)
prior = MultivariateNormal(mix.dim, cov=mix.added_noise_sq(tf))
x, _, nll = solve_flow(mix, prior, tf=tf)

plt.scatter(*x.T, s=1)
x1_cont = np.linspace(x[:, 0].min() - 1.0, x[:, 0].max() + 1.0, 200)
x2_cont = np.linspace(x[:, 1].min() - 1.0, x[:, 1].max() + 1.0, 200)
x_cont = np.stack(np.meshgrid(x1_cont, x2_cont), -1)
plt.contour(x_cont[:, :, 0], x_cont[:, :, 1], prior.density(x_cont), levels=10, alpha=0.5, cmap="plasma")
plt.title(f"NLL: {nll:.2f}")

In [ ]:
mix = VariancePreserving(norms)
prior = MultivariateNormal(mix.dim)
x, _, nll = solve_flow(mix, prior)

plt.scatter(*x.T, s=1)
x1_cont = np.linspace(x[:, 0].min() - 1.0, x[:, 0].max() + 1.0, 200)
x2_cont = np.linspace(x[:, 1].min() - 1.0, x[:, 1].max() + 1.0, 200)
x_cont = np.stack(np.meshgrid(x1_cont, x2_cont), -1)
plt.contour(x_cont[:, :, 0], x_cont[:, :, 1], prior.density(x_cont), levels=10, alpha=0.5, cmap="plasma")
plt.title(f"NLL: {nll:.2f}")

In [ ]:
mix = SubVariancePreserving(norms)
prior = MultivariateNormal(mix.dim)
x, _, nll = solve_flow(mix, prior)

plt.scatter(*x.T, s=1)
x1_cont = np.linspace(x[:, 0].min() - 1.0, x[:, 0].max() + 1.0, 200)
x2_cont = np.linspace(x[:, 1].min() - 1.0, x[:, 1].max() + 1.0, 200)
x_cont = np.stack(np.meshgrid(x1_cont, x2_cont), -1)
plt.contour(x_cont[:, :, 0], x_cont[:, :, 1], prior.density(x_cont), levels=10, alpha=0.5, cmap="plasma")
plt.title(f"NLL: {nll:.2f}")

### VE NLL per dim (solve_ivp)

In [ ]:
dims = [1, 2, 4, 8]
tf = 2.0
for dim in dims:
    norms = cube_vertices(dim)
    mix = VarianceExploding(norms)
    prior = MultivariateNormal(mix.dim, cov=mix.added_noise_sq(tf))
    x, _, nll = solve_flow(mix, prior, n_data=10000, tf=tf)
    print(f"dim: {dim}, nll: {nll}")

### VE NLL per dim (10, 20, 40, 80 steps)

#### Explicit Euler

In [ ]:
dims = [1, 2, 4, 8]
for dim in dims:
    norms = cube_vertices(dim)
    mix = VarianceExploding(norms)

    steps_list = [10, 20, 40, 80]
    for steps in steps_list:
        _, _, nll = solve_numerical_scheme(solver=EulerSolver, mix=mix, n_samples=10000, t_min=1e-6, tf=2.0, n_timesteps=steps, linear_ts=False)
        print("dim:", dim, "steps:", steps, "nll:", nll.mean())

    print('-'*5)

#### Implicit Euler (using Broyden method)

In [ ]:
dims = [1, 2, 4, 8]
for dim in dims:
    norms = cube_vertices(dim)
    mix = VarianceExploding(norms)

    steps_list = [10, 20, 40, 80]
    for steps in steps_list:
        _, _, nll = solve_numerical_scheme(solver=BroydenSolver, mix=mix, n_samples=10000, t_min=1e-6, tf=2.0, n_timesteps=steps, linear_ts=False)
        print("dim:", dim, "steps:", steps, "nll:", nll.mean())
    
    print('-'*5)

### Explicit Euler method issue illustration

In [ ]:
norms = cube_vertices(dim=2)

In [ ]:
mix = VarianceExploding(norms)
t_explicit, x_explicit, _ = solve_numerical_scheme(solver=EulerSolver, mix=mix, n_samples=1000, t_min=1e-6, tf=2.0, n_timesteps=10, linear_ts=False)

In [ ]:
mix = VarianceExploding(norms)
t_implicit, x_implicit, _ = solve_numerical_scheme(solver=BroydenSolver, mix=mix, n_samples=1000, t_min=1e-6, tf=2.0, n_timesteps=10, linear_ts=False)

In [ ]:
mix = VarianceExploding(norms)
prior = MultivariateNormal(mix.dim, cov=mix.added_noise_sq(2.0))
x_flow, x_init, _ = solve_flow(mix, prior, n_data=1000, t_min=0, tf=2.0)
x_flow = np.stack([x_init, x_flow], axis=1)

In [ ]:
def plot2d_clusters(x, side_len=1.0):
    fig, ax = plt.subplots(figsize=(5, 5))

    right = x[:, 0, 0] > 0.5 * side_len
    upper = x[:, 0, 1] > 0.5 * side_len
    colors = np.empty((len(x), 3))
    colors[right & upper] = mpl.color_sequences["tab20c"][16]
    colors[right & ~upper] = mpl.color_sequences["tab20c"][8]
    colors[~right & upper] = mpl.color_sequences["tab20c"][5]
    colors[~right & ~upper] = mpl.color_sequences["tab20c"][0]

    ax.scatter(*x[:, -1, :].T, s=2, c=colors)
    ax.set_xlim(-6, 10)
    ax.set_ylim(-6, 10)
    x_ticks = dict(bottom=False, top=False, labelbottom=False)
    y_ticks = dict(left=False, right=False, labelleft=False)
    ax.tick_params(**x_ticks, **y_ticks)
    ax.set_aspect("equal")

    # inset Axes....
    x1, x2, y1, y2 = -1.5, 1.5, -1.5, 1.5
    axins = ax.inset_axes(
        [0.5, 0.5, 0.45, 0.45],
        xlim=(x1, x2), ylim=(y1, y2), xticklabels=[], yticklabels=[])
    axins.scatter(*x[:, -1, :].T, s=30, c=colors)

    ax.indicate_inset_zoom(axins, edgecolor="black")

    return fig

#### Explicit Euler

In [ ]:
fig = plot2d_clusters(x_explicit)
# fig.savefig("clusters_euler_explicit.pdf", bbox_inches='tight')

#### Implicit Euler (using Broyden method)

In [ ]:
fig = plot2d_clusters(x_implicit)
# fig.savefig("clusters_euler_implicit.pdf", bbox_inches='tight')

#### Quasi-exact (solve_ivp)

In [ ]:
fig = plot2d_clusters(x_flow)
# fig.savefig("clusters_solve_ivp.pdf", bbox_inches='tight')